# Information Extraction - Assignment
This assignment is based on the Information Extraction lecture and the lab.


In [0]:
import nltk  as nk
nk.download("all")
import re
from statistics import mode

In [0]:
#inputfile="football_players.txt" #Location of the file

from google.colab import files
import io
uploaded = files.upload()

#To solve the error "initial_value must be unicode or None, not str" decode is used
inputfile = io.StringIO(uploaded['football_players.txt'].decode('utf-8')).readlines()


In [0]:
list_of_doc = [para.split("\n")[0] for para in inputfile]

list_of_doc = list(filter(None, list_of_doc)) #removing empty string
list_of_doc

In [0]:
list_of_doc[2]

# Task 1 (10 Marks)
Write a function that takes each document and performs:
1) sentence segmentation 2) tokenization 3) part-of-speech tagging

Please keep in mind that the expected output is a list within a list as shown below.


In [0]:
def ie_preprocess(document):
  sent_segment = nk.sent_tokenize(document)
  word_token = [nk.word_tokenize(sent) for sent in sent_segment]
  pos_sentences = [nk.pos_tag(sent) for sent in word_token]
  return pos_sentences

Run the following code to check your result for the first document (Ronaldo).

In [0]:
first_doc=list_of_doc[0]
pos_sent=ie_preprocess(first_doc)
pos_sent

Expected output
 [...[('He', 'PRP'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('forward', 'NN'),
  ('and', 'CC'),
  ('serves', 'NNS'),
  ('as', 'IN'),
  ('captain', 'NN'),
  ('for', 'IN'),
  ('Portugal', 'NNP'),
  ('.', '.')], ...]

# Task 2 (20 Marks)
Write a function that will take the list of tokens with POS tags for each sentence and returns the named entities (NE). 

Hint: Use binary=True while calling NE chunk function

In [0]:
def named_entity_finding(pos_sent):
  tree = nk.ne_chunk(pos_sent, binary=True)
  named_entities = []
  for subtree in tree.subtrees():    
    if subtree.label() == 'NE':
        entity = ""
        for leaf in subtree.leaves():
            entity = entity + leaf[0] + " "
        named_entities.append(entity.strip())
  return named_entities

pos_sents = ie_preprocess(list_of_doc[0])
named_entity_finding(pos_sents[0])
#There is typing error in the provided out Portuguese will come only once.

Expected output ['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

# Task 3 (10 Marks)

Now use the named_entity_finding() function to extract all NEs for each document.

Hint: pos_sents holds the list of lists of tokens with POS tags

In [0]:
def NE_flat_list_fn(pos_sents): 
    NE=[]
    for pos_sent in pos_sents:
        #Single line code here. Call the funtion named_entity_finding(pos_sent) and 
                #append the result to the NE list    
      NE.append(named_entity_finding(pos_sent))
    #Single line code here. Flatten the list of lists to the single list NE_flat_list
    NE_flat_list = [item for sublist in NE for item in sublist]
    return NE_flat_list

In [0]:
NE_flat_list_fn(pos_sents)


# Task 4 (40 Marks)

Write functions to extract the name of the player, country of origin and date of birth as well as the following relations: team(s) of the player and position(s) of the player.

Hint: Use the re.compile() function to create the extraction patterns

Reference: https://docs.python.org/3/howto/regex.html

In [0]:
#Function to remove the partial matches.
#For e.g. ['Barcelona', 'Paris Saint-Germain', 'Juventus', 'FC Barcelona', 'Ajax', 'Inter Milan', 'A.C. Milan']
#This function will remove the "Barcelona" as Barcelona and FC Barcelona is same. Moreover, Barcelona word is coming inside the FC Barcelona.

def filtering_partial_regex(teams_ans):
  single_words = []
  not_single = []
  for xx in teams_ans:
    if(len(xx.split(" ")) == 1):
      single_words.append(xx)
    else:
      not_single.append(xx)

  for y in not_single:
    for r in y.split(" "):
      if("-" in r):
        [teams_ans.remove(t) for t in teams_ans if t==r.split("-")[1]]
        [teams_ans.remove(t) for t in teams_ans if t==r.split("-")[0]]
      else:
        if(r in single_words):
          [teams_ans.remove(t) for t in teams_ans if t==r]
  
  return(teams_ans)

In [0]:
def name_of_the_player(doc):
    name =[]
    preprocessed_words = ie_preprocess(doc)
    list_to_pass = named_entity_finding(preprocessed_words[0])
    
    #The pattern of palyer name is at the starting of the paragraph till the born word. Making the generalised regex which is given below. 
    NAME = re.compile(r'(?=^)(\w.+)(?=\,\s(C|O))|(?=^)(\w.+)(?=\s\(\bborn\b)')
    name = NAME.search(doc).group()
    return name

def country_of_origin(doc):
    #For the country the pattern is a word preceding to the national team. The generalised regex is working in a same way. Other possible regex is also given below (commented) which will also work similarly.
    #COUNTRY = re.compile(r'(?<=\b(?<=\band\b|\bfor\b) the\s\b).[\w]+(?=\b national team\b)')
    COUNTRY = re.compile(r'(\w+)(?=\s\bnational team\b)', re.IGNORECASE)
    
    country = COUNTRY.search(doc).group()    
    return country

def date_of_birth(doc):
    #For the born the pattern is born and details in the brackets. The generalised regex is finding the born and its details enclosed in brackets and displaying the result.
    #Another alternative regex is also mentioned just below (commented) which also gives the correct result.
    #\bborn*[a-z|A-Z|\d|\s]+\b
    BORN = re.compile(r'(?<=\bborn\s\b).[a-z|A-Z|\d|\s]+', re.IGNORECASE)
    
    date = BORN.search(doc).group()    
    return date

def team_of_the_player(doc):
    #NOTE: Considering players team as a club only not the national teams as we have already taken out the national teams in the country_of_origin method.
    #If national team is also part of the team than we can combine the result of this method with country_of_origin. eg. final_result_team + country_of_origin(doc).
    result_team = list()
    teams = list()
    
    #For finding the team of the players making almost generalised regex which is given below.
    #The main idea is using the keywords like "plays for", "siging with", "moved to", "transferred to", "clubs" etc. These are some of the normal and generalised words which will be present in
    #any of the document. Here, modify them with the help of grammer. We can also use POS tagging and grammer for parsing, but since question asked for the regex so building the regex.
    TEAM = re.compile(r'(?<=\bclub\s\b)((\s*[A-Z]\w+){0,2})|(?:\bsigning with\s\b|\bplaying for\s\b|(?<=\b\,\sbefore moving to\s\b)|\bseasons at the\s\b|\btransferred to\s\b|\bsigned (?:\bby\s\b)\b)((\s*\w+){0,2})|(?<=\bclubs\s\b)(\s*\w.+)(?=\.\s\bHe\b)|(?<=\byouth team of\s\b)((\s*[A-Z]\w+))|(?<=\bplayed for\s\b)(\s*\w.+)(?=\,\s\band the(?:\s\w+)\snational team\b)|(?<=\bprofessional career at\s\b)(\s*\w+)|(?<=\bas the captain for\s\b)(\w+)|(?<=\bsigned for rival side\s\b)((\s*\w+){0,2})|(?:\bHe joined\s\b|\bjoining\s\b)(\w.+)(?=\s\bin a\b)|(?<=\bHe joined\s\b)((\s*\w+|\-){0,4})|(?<=\bsigned for\s\b)(\w.+)(?=\s\band excelled\b)|(?<=\bhe moved to\s\b)(\s*\w+)|(?<=\bmoved to\s\b)((\s*\w+){0,2})|(?<=\bto play for\s\b)(\s*\w.+)(?=\s\bbefore\b)|(?<=\bto play for\s\b)(\w+)')
    
    team = TEAM.findall(doc)    
    for i in range(len(team)):
      teams = teams + list(filter(None, team[i]))
    
    #Creating the regex for removing extra space from the words
    partial_match_words = re.compile(r'^\s\b')   
    
    #Loop which remove the "commans" and "and" in the result so that we will get list of all the teams.
    #For e.g. from "Manchester United, Preston North End, Real Madrid, Milan, LA Galaxy, Paris Saint-Germain" all the "," and "and" will be removed.
    for p in range(len(teams)):
      if(',' in teams[p]):
        removing_commas = teams[p].split(", ")
        for rc in range(len(removing_commas)):
          if(" and " in removing_commas[rc]):
            result_team = result_team + removing_commas[rc].split(" and ")
          else:
            result_team.append(removing_commas[rc])
      elif('and' in teams[p]):
          result_team = result_team + teams[p].split(" and ")
      elif(partial_match_words.match(teams[p]) == None):
        result_team.append(teams[p])
    
    final_result_team = filtering_partial_regex(list(set(result_team)))
    
    return list(set(final_result_team))

def position_of_the_player(doc):
    result_pos = list()
    
    #Since in soccer we have limited positions, so creating the list of all the positions and finding it in the given document. The generalised regex is given below.
    POSITION = re.compile(r'(\bstriker\b)|(\bforward\b)|(\bleft winger\b|\bright winger\b|winger)|(\bcentral midfielder\b|\battacking midfielder\b)|((\b(central\s)*(left-back|right-back)\b)+)|(\b(full|central) back\b)|(\bgoalkeeper\b)', re.IGNORECASE)
    position = POSITION.findall(doc)
    
    #Removing the empty tupples from the above output.
    for i in range(len(position)):
      result_pos = result_pos + list(filter(None, position[i]))
      
    return list(set(result_pos))

In [0]:
name_of_the_player(list_of_doc[9])

In [0]:
country_of_origin(list_of_doc[9])

In [0]:
position_of_the_player(list_of_doc[8])

In [0]:
team_of_the_player(list_of_doc[5])

Execute the below command to check your fuction


In [0]:
date_of_birth(list_of_doc[2])

Expected output '5 February 1992'

# Task 5 (10 Marks)

Write a function using the outputs from the previous functions to generate JSON-LD output as follows.

Reference: https://json-ld.org/primer/latest/

{ "@id": "http://my-soccer-ontology.com/footballer/name_of_the_player",

    "name": "",
    "born": "",
    "country": "",
    "position": [
        { "@id": "http://my-soccer-ontology.com/position",
            "type": ""
        }
     ]   
     "team": [
        { "@id": "http://my-soccer-ontology.com/team",
            "name": ""
        }   
     ]
}


In [0]:
import json

In [0]:
#It will take a document as an argument. Passing only one argument so that user don't have to pass details manually (for user simplicity).
def generate_jsonld(doc):
    #Hint: arg1,arg2,..... are the arguments you will be passing to the function
    
    #Taking out the required document.
    player_name = name_of_the_player(doc)
    player_dob = date_of_birth(doc)
    player_country = country_of_origin(doc)
    player_position = []
    player_teams = []
    
    #Converting the player all position into JSON-LD format
    player_all_position = position_of_the_player(doc)
    for pos in range(len(player_all_position)):
      individual_pos = {
             "@id": "http://my-soccer-ontology.com/"+ player_all_position[pos],
             "type": player_all_position[pos]
            }
      player_position.append(individual_pos)

    #Converting the player all teams into JSON-LD format
    player_all_team = team_of_the_player(doc)
    for team in range(len(player_all_team)):
      individual_team = {
             "@id": "http://my-soccer-ontology.com/"+ player_all_team[team],
             "type": player_all_team[team]
            }
      player_teams.append(individual_team)

    json_data = { "@id": "http://my-soccer-ontology.com/footballer/"+ player_name,

                  "name": player_name,
                  "born": player_dob,
                  "country": player_country,
                  "position": player_position, 
                   "team": player_teams
                  }
    return json.dumps(json_data)


In [0]:
generate_jsonld(list_of_doc[3])

# Task 6 (10 Marks)
Identify one other relation (besides team and player) and write a function to extract this. Also extend the JSON-LD output accordingly.

In [0]:
#========================================================== Solution ============================================================================



---


I am taking debut year and the age of the player from which he enters into professional football career. However, it is important to note that, these two values are not available for all the players. Hence, if debut year or debut age is not present I have added as "Not Available" as a default value.

---





In [0]:
def career_begin(doc):
  career_result = list()
  
  #Regex will match the pattern (checking the keywords like age in, debut in etc) and fecth teh age and year of the player. 
  #For e.g. In case of Cristiano Ronaldo it will give "at age 18 in 2003" as a result.
  CAREER = re.compile(r'(\bat age\s\b|\b(debut|\d+)* aged\s\b|\bdebut in\s\b)(.\d+)(.in.\w+.\d+)*')  
  career = CAREER.search(doc)
  
  #Checking if the values (age and year) are present or not. If not, adding the NOt Available as a default value.
  if(career == None):
    age = debut_year = "Not Available"
  else:
    age = re.search(r'\d{2}', career.group())
    age = age.group() if age != None else "Not Available"
    debut_year = re.search(r'\d{4}', career.group())
    debut_year = debut_year.group() if debut_year != None else "Not Available"
  
  career_result.extend((age, debut_year))
  return career_result

In [0]:
career_begin(list_of_doc[0])

In [0]:
#This is the updated function containing the Debut age and year details.
#It will take a document as an argument. Passing only one argument so that user don't have to pass details manually (for user simplicity).
def new_generate_jsonld(doc):
    #Code goes here
    #Hint: arg1,arg2,..... are the arguments you will be passing to the function
    player_name = name_of_the_player(doc)
    player_dob = date_of_birth(doc)
    player_country = country_of_origin(doc)
    player_position = []
    player_teams = []
    player_debut_age = career_begin(doc)[0]
    player_debut_year = career_begin(doc)[1]

    #Converting the player all position into JSON-LD format
    player_all_position = position_of_the_player(doc)
    for pos in range(len(player_all_position)):
      individual_pos = {
             "@id": "http://my-soccer-ontology.com/"+ player_all_position[pos],
             "type": player_all_position[pos]
            }
      player_position.append(individual_pos)

    #Converting the player all teams into JSON-LD format
    player_all_team = team_of_the_player(doc)
    for team in range(len(player_all_team)):
      individual_team = {
             "@id": "http://my-soccer-ontology.com/"+ player_all_team[team],
             "type": player_all_team[team]
            }
      player_teams.append(individual_team)

    json_data = { "@id": "http://my-soccer-ontology.com/footballer/"+ player_name,

                  "name": player_name,
                  "born": player_dob,
                  "country": player_country,
                  "position": player_position, 
                  "team": player_teams,
                  "debut year": player_debut_year,
                  "debut age": player_debut_age,
                  }
    return json.dumps(json_data)


In [0]:
new_generate_jsonld(list_of_doc[0])